In [3]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, Aer
import random
import networkx as nx
import matplotlib.pyplot as plt
import arviz as azr
import random
from qiskit import QuantumCircuit
from qiskit.circuit.library import RYGate
from qiskit.circuit.library import XGate
from qiskit.circuit import ControlledGate

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
#------------------------------- read in the phylogenetic graph----------------------------------------------
f = open('Salmonella.net.nex')
text = f.readlines()
section=0
section_1=[]
section_2=[]
section_3=[]
section_4=[]

for line in text:
    
    if line[:-1]==';':   
        section=0
    
    if section==1:
        section_1.append(line[:-1])
    elif section==2:
        section_2.append(line[:-1])
    elif section==3:
        section_3.append(line[:-1])
    elif section==4:
        section_4.append(line[:-1])
    if line[:-1]=='TRANSLATE':
        section=1
        print ('change to' +str(section))
    elif line[:-1]=='VERTICES':
        section=2
        print ('change to' +str(section))
    elif line[:-1]=='VLABELS':   
        section=3
        print ('change to' +str(section))
    elif line[:-1]=='EDGES':
        section=4
        print ('change to' +str(section))

f.close()


change to1
change to2
change to3
change to4


In [5]:
# Parameters about phylogenetic tree.

n_vertices=3313
n_edges=5945
n_taxa=248
n_traits=1

In [6]:
# ----------------Usefull lists and dicts about graph properties-----------------------


# --------------------------------------------------------------------
taxa_vertices=[] # list of all 248 labels of observed taxa vertices. The vertices are labeled computationally e.g. starting from 0.
taxa_label={} # dicts of taxa: {label : name}  ;   example:  '2': 'H88_2002' .  The vertices are not labeled computationally e.g. starting from 1.
reverse_taxa_label={} # reverse dicts of taxa_label: {name : label};  example: 'H88_2002' :'2'.  The vertices are not labeled computationally e.g. starting from 1.

for line in section_1:
    ob=line.split(' ')
    taxa_label[ob[0]]=ob[1][1:-2]
    reverse_taxa_label[ob[1][1:-2]]=int(ob[0])
    taxa_vertices.append(int(ob[0])-1)
    
# --------------------------------------------------------------------    
non_taxa_vertices=[] # list of all (3313-248) labels of non-taxa vertices. The vertices are labeled computationally e.g. starting from 0.
ind=0
for i in range(n_vertices):
    if (ind>247 or i!=taxa_vertices[ind]):
        non_taxa_vertices.append(i)
    else:
        ind+=1
#  ----------------------------------------------------------------------     
edges=[] # list of all edges ([i,j] for i<j). The vertices i,j are not labeled computationally e.g. starting from 1.
edges_com_base=[]  # # list of all edges([i,j] for i<j). The vertices are not labeled computationally e.g. starting from 1.
for line in section_4:
    ob=line.split(' ')
    edges.append([ob[1],ob[2]]) 
    edges_com_base.append([int(ob[1])-1,int(ob[2])-1])

#------------------------------------------------------------------------
vertice_neighbors=[] #list of lists, vertice_neighbors[i]=[i's neighbors on the phylogenetic graph]. The vertices are labeled computationally e.g. starting from 0.
neighbor_num=[] #list of intergers, neighbor_num[i]= neighber counts of i. The vertices are labeled computationally e.g. starting from 0.
for i in range (n_vertices):
    vertice_neighbors.append([])
for edge in edges_com_base:
    vertice_neighbors[edge[0]].append(edge[1])
    vertice_neighbors[edge[1]].append(edge[0])

for i in range(len(vertice_neighbors)):
    neighbor_num.append(len(vertice_neighbors[i]))
    
    
#---------------------------------------------------------------------------
traits_taxa={} #  {name : 4 bits bitstring} dicts of the observed binary informations on first 4 traits of a given taxa. Example:  'H88_2002': '1100'.
f = open('AMR.xml')
text = f.readlines()
for i in range(248):
    traits_taxa[text[1760+4*i].split("\"") [1]]=text[1760+4*i+1][3:7]
f.close()

In [7]:

#  Usefull functions

# --------- Input: target vertex, state about a specific trait(=state[trait_label]) , list of neighbors----------
# --------- Output: 8+ (Sum over the multiplications of bit information among target vertex and all of its neighbors), note that output >=0 since deg(G)=8.
def local_relation(vertex,state_of_trait,neighbor):
    E=8
    if vertex>=0:
        for i in neighbor[vertex]:
            E+=state_of_trait[vertex]*state_of_trait[i]
#     theta=2*np.arcsin(np.exp(-J*E))-np.pi
    return E

def general_local_relation(Jump,state,vertice_neighbors):
    logprob_local=0
    for i in range(len(Jump)):
        logprob_local+=local_relation(Jump[i][1],state[Jump[i][0]],vertice_neighbors)
        if Jump[i][1]>=0:
            state[Jump[i][0]][Jump[i][1]]=state[Jump[i][0]][Jump[i][1]]*(-1)
            
    for i in range(len(Jump)):
        if Jump[len(Jump)-i-1][1]>=0:
            state[Jump[len(Jump)-i-1][0]][Jump[len(Jump)-i-1][1]]=state[Jump[len(Jump)-i-1][0]][Jump[len(Jump)-i-1][1]]*(-1)
        
    return logprob_local
  

# --------- Input: 1. list of edges amd target state / Output: log_prob(Ising Model) of this state-----------

def log_prob(edges,state):
    H=0
    for trait in range(len(state)):
        state_of_trait=state[trait]
        for edge in edges:
            H+=state_of_trait[int(edge[0])]*state_of_trait[int(edge[1])]
    return H


# --------- Input: taxa,trait (label)  / Output: bit information of the observed taxa +1/-1.----------------
def taxa_trait(taxa,trait):
    return(1-2*int(traits_taxa[taxa_label[str(taxa+1)]][trait]))



# ----------- Input: number of traits & vertices & taxa/ Output: random function decide to stay at original state or flip arbitrary bit //// This function is used to constuct \bar(q) described in section 4.
def stay_or_flip(n_traits,n_vertices,n_taxa):
    signal=random.randrange(n_traits*(n_vertices-n_taxa)+1)
    if signal==0:
        return 'stay'
    else:
        return 'flip'
    
    
def stay_or_flipbit(n_traits,n_vertices,n_taxa):
    
    Proposal_trait=0
    Proposal_vertex=-1
    
    signal=random.randrange(n_traits*(n_vertices-n_taxa)+1)
    if signal>=1:
        Proposal_trait=random.randrange(n_traits)
        Proposal_vertex=non_taxa_vertices[random.randrange(n_vertices-n_taxa)]
        
    return [Proposal_trait,Proposal_vertex]

def lenbit_remain(n,i):
    return ((bin(2**n)[2:]+bin(i)[2:])[-n:])

In [8]:
 def reruntype_weighted_sample(p_list,accept_list):
    accept_or_not=0
    while accept_or_not==0:
        p_hat=random.choices(p_list, k=1)[0]
        accept_or_not=random.choices([0,1], weights=[1-accept_list[p_hat],accept_list[p_hat]], k=1)[0]
    return p_hat

In [9]:
#  


def QPMCMC2_iteration_general(vertice_neighbors,state,n_vertices,n_traits,n_Props,J,fliptimes):
    
#     c_qubits,label_qubits, control_list, full_qubit_list, options, bi_repre_options, theta_list = Quantum_Param

# ----------do first jump--------------------------------
    first_jump=[]
    for i in range (fliptimes):
         first_jump.append(stay_or_flipbit(n_traits,n_vertices,n_taxa))      

    for i in range(fliptimes):
        if first_jump[i][1]>=0:
            state[first_jump[i][0]][first_jump[i][1]]=state[first_jump[i][0]][first_jump[i][1]]*(-1)
        
    #     print([Proposal_trait,Proposal_vertex])
    
# ----------generate second jump proposal list -----------------------------

    # ----------(case 1, p=0)-----------------
    Jump_List=[first_jump]  
    log_weight=[general_local_relation(first_jump,state,vertice_neighbors)] #if p=0, second jump shoud jump back
    # ----------(case 2, p>=1)-----------------    
    for p in range(n_Props):
        
        second_jump=[]
        for i in range(fliptimes):
             second_jump.append(stay_or_flipbit(n_traits,n_vertices,n_taxa))  
        Jump_List.append(second_jump)        
        log_weight.append(general_local_relation(second_jump,state,vertice_neighbors))
    
# ----------select one of the proposal(weighted)--------------------- 
    weight=[] #list of \pi^*_p
    for p in range(n_Props+1):
        weight.append(np.exp(-2*J*log_weight[p])) 
#        ---------rerun test in QPMCMC2-----
    R=np.mean(weight)
    rerun=0
    run_count=0
    while (rerun ==0):   
        run_count+=1    
        rerun=random.choices([0,1],weights=[1-R,R],k=1)[0]
#        ---------end rerun test QPMCMC2-----        
    
    p_List=[]
    for p in range(n_Props+1):
        p_List.append(p)
    p_chosen=random.choices(p_List, weights=weight, k=1)[0]
        
        
        
#     quantum version:
#     p_chosen,rerun=Quantum_Weighted_Sampling(log_weight, n_Props ,fliptimes, options, bi_repre_options, theta_list, full_qubit_list , control_list , label_qubits , c_qubits)

    
# ----------do second jump------------------------------------------------

    for i in range(fliptimes):
        if Jump_List[p_chosen][i][1]>=0:
            state[Jump_List[p_chosen][i][0]][Jump_List[p_chosen][i][1]]=state[Jump_List[p_chosen][i][0]][Jump_List[p_chosen][i][1]]*(-1)
        
    return [state,run_count]

In [10]:
def QPMCMC2(start_or_conti,J,n_iters,n_Props,fliptimes,add_nameinfo):
    
# -------------------Quantum Circuits Parameters-------------------------------

#     c_qubits=int(np.ceil(np.log2(2*8*fliptimes+1)))
#     label_qubits=int(np.ceil(np.log2(n_Props+1)))


#     control_list=[]
#     for i in range(c_qubits+1):
#         control_list.append(label_qubits+i)
#     full_qubit_list=[]    
#     for i in range(label_qubits+c_qubits+1):
#         full_qubit_list.append(i)

#     options=[]
#     bi_repre_options=[]
#     theta_list=[]
#     for i in range(2*8*fliptimes+1):
#         options.append(i)
#         bi_repre_options.append(lenbit_remain(c_qubits,i))
#         theta_list.append(2*np.arcsin(np.exp(-J*i)))
        
#     Quantum_Param=[c_qubits,label_qubits, control_list, full_qubit_list, options, bi_repre_options, theta_list]
        
    

# ---------- naming this sampling process with P, J, number of kinds of trait m, and addtional self define info.------------  

    Barker_MCMC_name='Barker_flips='+str(fliptimes)+'_P='+str(n_Props)+'_J='+str(J)+'_m='+str(n_traits)+add_nameinfo
    print('starting : '+Barker_MCMC_name)
    
# ---------- initial state preparation: 1)continue with saved mcmc process or 2) start from a new one-------------- 

    state=[]
    if start_or_conti=='start':
        for trait in range(n_traits):
            state_of_trait=[]
            for vertex in range(n_vertices):
                if vertex in non_taxa_vertices:
                    state_of_trait.append((-1)**(vertex))
                else:
                    state_of_trait.append(taxa_trait(vertex,trait))
            state.append(state_of_trait)
            
    elif start_or_conti=='conti':    
        print('starting : '+Barker_MCMC_name)
        f = open('tree_history_'+Barker_MCMC_name+'.txt')
        text = f.readlines()    
        for trait in range(n_traits): 
            state_of_traits=text[-n_traits+trait].split(' ')
            for lattices in range(len(state_of_traits)):
                state_of_traits[lattices]=int(state_of_traits[lattices])
            state.append(state_of_traits)
        f.close()
    else:
        print("Initial_preparation_false")
    
    
# ---------- Start MCMC iterations-------------------------------------------  
    log_prob_history_block=[]
    rerun_history_block=[]
    
    for ite in range(n_iters):
#         print('start '+str(ite))
        state_sampled,reruntimes=QPMCMC2_iteration_general(vertice_neighbors,state,n_vertices,n_traits,n_Props,J,fliptimes).copy()
    
        log_pb=log_prob(edges_com_base,state_sampled)
        if (ite%500==0):
            print(str(ite))
            print(log_pb)
        log_prob_history_block.append(log_pb)
        rerun_history_block.append(reruntimes)
        state=state_sampled
        
        #-----save a sampled state + save all 500 log_prob for every 500 iterations  
        if (ite%500==499) :
            with open('logprob_'+Barker_MCMC_name+'.txt', 'a') as file:
                for j in range(len(log_prob_history_block)):
                    file.write(str(log_prob_history_block[j])+'\n')
            with open('rerun_'+Barker_MCMC_name+'.txt', 'a') as file:
                for j in range(len(rerun_history_block)):
                    file.write(str(rerun_history_block[j])+'\n')
            for trait in range(n_traits) :
                with open('tree_history_'+Barker_MCMC_name+'.txt', 'a') as file:
                    state_str = ' '.join(map(str, state[trait]))
                    file.write(state_str+'\n')

            log_prob_history_block=[]

In [11]:
def MH_iteration_jumptwice_general(vertice_neighbors,state,n_vertices,n_traits,J,fliptimes):
    
# ---------- generate first and second jump plan: two pairs of random trait/ random non-taxa vertex------------------------

    first_jump=[]
    for i in range (fliptimes):
        first_jump.append(stay_or_flipbit(n_traits,n_vertices,n_taxa))      
            
            
    second_jump=[]
    for i in range(fliptimes):
        second_jump.append(stay_or_flipbit(n_traits,n_vertices,n_taxa))  

    
# -----------do first jump--------------------------------------    
    for i in range(fliptimes):
        if first_jump[i][1]>=0:
            state[first_jump[i][0]][first_jump[i][1]]=state[first_jump[i][0]][first_jump[i][1]]*(-1)
    
# ----------calculaterelative probability \frac{\pi'}{\pi_0}-----------------------------------


    log_relative_possibility=general_local_relation(second_jump,state,vertice_neighbors)-general_local_relation(first_jump, state,vertice_neighbors)
    relative_possibility=np.exp(-2*J*log_relative_possibility)
    
# ----------accept rate= min(1,\frac{\pi'}{\pi_0})----------------------------------------

    if relative_possibility>=1:
        relative_possibility=1
        
# ----------accept/reject decision + second jump(accept: jump another time; reject:jump back )-------------------------------------------------- 

    p_chosen=random.choices([0,1], weights=[1-relative_possibility,relative_possibility], k=1)[0]
    
    if p_chosen==0:
        for i in range(fliptimes):
            if first_jump[i][1]>=0:
                state[first_jump[i][0]][first_jump[i][1]]=state[first_jump[i][0]][first_jump[i][1]]*(-1)
    elif  p_chosen==1:
        for i in range(fliptimes):
            if second_jump[i][1]>=0:
                state[second_jump[i][0]][second_jump[i][1]]=state[second_jump[i][0]][second_jump[i][1]]*(-1)
    else:
        return False
    
    return state
    

In [12]:
def MH_sampling_general(start_or_conti,J,n_iters,fliptimes,add_nameinfo):
    
# ---------- naming this sampling process with P, J, number of kinds of trait m, and addtional self define info.------------  

    MH_MCMC_name='MetropolisHastings_flips='+str(fliptimes)+'_J='+str(J)+'_m='+str(n_traits)+add_nameinfo

    print('starting : '+MH_MCMC_name)
    
# ---------- initial state preparation: 1)continue with saved mcmc process or 2) start from a new one--------------    
    
    state=[]
    if start_or_conti=='start':
        for trait in range(n_traits):
            state_of_trait=[]
            for vertex in range(n_vertices):
                if vertex in non_taxa_vertices:
                    state_of_trait.append((-1)**(vertex))
                else:
                    state_of_trait.append(taxa_trait(vertex,trait))
            state.append(state_of_trait)
            
    elif start_or_conti=='conti':
        
        print('starting : '+MH_MCMC_name)
        f = open('tree_history_'+MH_MCMC_name+'.txt')
        text = f.readlines()    
        for trait in range(n_traits): 
            state_of_traits=text[-n_traits+trait].split(' ')
            for lattices in range(len(state_of_traits)):
                state_of_traits[lattices]=int(state_of_traits[lattices])
            state.append(state_of_traits)
        f.close()
    else:
        print("Initial_preparation_false")
    
# ---------- Start MCMC iterations--------------    

    log_prob_history_block=[] #save sampled states' log_prob  

    for i in range(n_iters):
#         print('start '+str(i))
        a=MH_iteration_jumptwice_general(vertice_neighbors,state,n_vertices,n_traits,J,fliptimes).copy()

        
        log_pb=log_prob(edges_com_base,a)
        print(log_pb)
        log_prob_history_block.append(log_pb)

        state=a
        if (i%500==499):
            print('start '+str(i))
            with open('logprob_'+MH_MCMC_name+'.txt', 'a') as file:
                for j in range(len(log_prob_history_block)):
                    file.write(str(log_prob_history_block[j])+'\n')

            for trait in range(n_traits) :
                with open('tree_history_'+MH_MCMC_name+'.txt', 'a') as file:
                    state_str = ' '.join(map(str, state[trait]))
                    file.write(state_str+'\n')

            log_prob_history_block=[]

In [14]:
# for repetition in [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]:
#     MH_sampling_general(start_or_conti="start",J=0.03,n_iters=120000,fliptimes=1,add_nameinfo="_r"+str(repetition)+"_esscountingmean")

In [ ]:
for repetition in range(10):
    for p in [1,3,5,10, 30, 50, 70]:
        QPMCMC2(start_or_conti="conti",J=0.03,n_iters=120000-12000,n_Props=p,fliptimes=1,add_nameinfo="_r"+str(repetition)+"_esscountingmean")